In [47]:
# pip install pyodbc

In [66]:
import pandas as pd
import pyodbc
import os

In [67]:
# Define the parameters for connect to SQL Server database
driver_name = "ODBC Driver 17 for SQL Server"
server_name = "TIT-20230930FXD\\SQLEXPRESS"
database_name = "brazilian_ecommerce" # using database_name = "master" for create new database
trusted_connection = "yes"

# Define connection string
conn_str = f"DRIVER={driver_name};SERVER={server_name};DATABASE={database_name};Trusted_Connection={trusted_connection}"

In [68]:
# Read CSV file
data_path = r"C:\Users\Admin\Documents\GitHub\data-science-bootcamp9\09_Essential Python for Data Analyst\Project - Automatically Create and add data to a SQL Server\data"
file_name = "olist_order_items_dataset.csv"
df = pd.read_csv(os.path.join(data_path, file_name), encoding="utf8")
df.info()

order_id                object
order_item_id            int64
product_id              object
seller_id               object
shipping_limit_date     object
price                  float64
freight_value          float64
dtype: object

In [69]:
df.duplicated().sum()

0

In [70]:
df.drop_duplicates(inplace=True)

In [71]:
df.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14


In [72]:
# Convert "review_answer_timestamp" column to datetime type
df["shipping_limit_date"] = pd.to_datetime(df["shipping_limit_date"])

In [73]:
df = df.astype(object)
df.dtypes

order_id               object
order_item_id          object
product_id             object
seller_id              object
shipping_limit_date    object
price                  object
freight_value          object
dtype: object

In [74]:
# Convert NaN values to None
df = df.where(pd.notnull(df), None)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112650 entries, 0 to 112649
Data columns (total 7 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   order_id             112650 non-null  object
 1   order_item_id        112650 non-null  object
 2   product_id           112650 non-null  object
 3   seller_id            112650 non-null  object
 4   shipping_limit_date  112650 non-null  object
 5   price                112650 non-null  object
 6   freight_value        112650 non-null  object
dtypes: object(7)
memory usage: 6.0+ MB


In [75]:
df.nunique()

order_id               98666
order_item_id             21
product_id             32951
seller_id               3095
shipping_limit_date    93318
price                   5968
freight_value           6999
dtype: int64

In [76]:
print(df.order_id.astype(str).str.len().max())
print(df.product_id.astype(str).str.len().max())
print(df.seller_id.astype(str).str.len().max())

32
32
32


In [77]:
# drop_table_query = """
#     DROP TABLE IF EXISTS order_items
# """

# create_table_query = """
#     CREATE TABLE order_items(    
#         order_id VARCHAR(50) NOT NULL,
#         order_item_id INT NOT NULL,
#         product_id VARCHAR(50) NOT NULL,
#         seller_id VARCHAR(50) NOT NULL,
#         shipping_limit_date DATETIME NULL,
#         price FLOAT NULL,
#         freight_value FLOAT NULL
#     );
# """

# with pyodbc.connect(conn_str) as con:
#     con.execute(drop_table_query)
#     con.execute(create_table_query)
#     con.commit()

In [78]:
# Define SQL command for truncate table
truncate_query = """
    TRUNCATE TABLE order_items
"""

# Execute command SQL for truncate table
with pyodbc.connect(conn_str) as con:
    cursor = con.cursor()
    cursor.execute(truncate_query)
    con.commit()

In [79]:
with pyodbc.connect(conn_str) as con:
    cursor = con.cursor()
    for index, row in df.iterrows():
        insert_query = """
            INSERT INTO order_items
            VALUES (?, ?, ?, ?, ?, ?, ?)
        """
        # Execute command SQL for insert new data
        cursor.execute(insert_query, tuple(row))
    con.commit()

In [80]:
with pyodbc.connect(conn_str) as con:
    df_query = pd.read_sql("SELECT * FROM order_items", con)
    
df_query.info()

C:\Users\Admin\AppData\Local\Temp\ipykernel_2024\3269614454.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_query = pd.read_sql("SELECT * FROM order_items", con)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112650 entries, 0 to 112649
Data columns (total 7 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   order_id             112650 non-null  object        
 1   order_item_id        112650 non-null  int64         
 2   product_id           112650 non-null  object        
 3   seller_id            112650 non-null  object        
 4   shipping_limit_date  112650 non-null  datetime64[ns]
 5   price                112650 non-null  float64       
 6   freight_value        112650 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(1), object(3)
memory usage: 6.0+ MB
